In [9]:
import pyswip, ctypes

class PrologMT(pyswip.Prolog):
    """Multi-threaded (one-to-one) pyswip.Prolog ad-hoc reimpl"""
    _swipl = pyswip.core._lib

    PL_thread_self = _swipl.PL_thread_self
    PL_thread_self.restype = ctypes.c_int

    PL_thread_attach_engine = _swipl.PL_thread_attach_engine
    PL_thread_attach_engine.argtypes = [ctypes.c_void_p]
    PL_thread_attach_engine.restype = ctypes.c_int

    @classmethod
    def _init_prolog_thread(cls):
        pengine_id = cls.PL_thread_self()
        if (pengine_id == -1):
            pengine_id = cls.PL_thread_attach_engine(None)
            print("{INFO} attach pengine to thread: %d" % pengine_id)
        if (pengine_id == -1):
            raise pyswip.prolog.PrologError("Unable to attach new Prolog engine to the thread")
        elif (pengine_id == -2):
            print("{WARN} Single-threaded swipl build, beware!")

    class _QueryWrapper(pyswip.Prolog._QueryWrapper):
        def __call__(self, *args, **kwargs):
            PrologMT._init_prolog_thread()
            return super().__call__(*args, **kwargs)

In [13]:
# For each Prolog() instance, a new thread is created
from threading import Thread

def run_prolog_thread():
    prolog = PrologMT()
    prolog.consult("knowledge_base1.pl")
    print(list(prolog.query("father(michael,X)")))

def run_prolog_thread2():
    prolog = PrologMT()
    prolog.consult("knowledge_base2.pl")
    print(str(list(prolog.query("father(peter,X)"))) + 'thread2')


thread = Thread(target=run_prolog_thread)
thread2 = Thread(target=run_prolog_thread2)
thread.start()
thread2.start()
thread.join()
thread2.join()

{INFO} attach pengine to thread: 20
{INFO} attach pengine to thread: 21
[{'X': 'bob'}, {'X': 'brian'}]thread2
[{'X': 'alison'}]


In [13]:
from multipledispatch import dispatch

class A:
    @dispatch(int)
    def __init__(self, x):
        print("int")
        self.x = x
        
    @dispatch(str)
    def __init__(self, x):
        print("str")
        self.x = x

A(1)
A("1")

int
str


In [2]:
from cripthmik.solve import GenerateAndTest, Enumerate, ConstraintProgramming
from cripthmik.utils import Cryptarithm

In [5]:
cryptarithm = Cryptarithm("SEND + MORE = MONEY")
cryptarithm = Cryptarithm("a * B = C", case_sensitive=True)

In [6]:
solver = ConstraintProgramming()
print(list(solver.solve(cryptarithm)))
# %timeit list(solver.solve(cryptarithm))

PrologError: Caused by: 'solve([[a], *, [B], =, [C]]), label([B,a,C])'. Returned: 'error(type_error(integer, a), _11484)'.

In [7]:
solver = GenerateAndTest()
print(list(solver.solve(cryptarithm)))
# %timeit list(solver.solve(cryptarithm))

[]


KeyboardInterrupt: 

In [8]:
solver = Enumerate()
print(list(solver.solve(cryptarithm)))
# %timeit list(solver.solve(cryptarithm))

[{'C': 6, 'B': 2, 'a': 3}, {'C': 6, 'B': 3, 'a': 2}, {'C': 8, 'B': 2, 'a': 4}, {'C': 8, 'B': 4, 'a': 2}]


KeyboardInterrupt: 